<a href="https://colab.research.google.com/github/Wushuyen/RetailCompany/blob/main/%E8%B3%87%E6%96%99%E6%94%B6%E9%9B%8620_%E5%B7%AB%E5%8F%94%E6%99%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import datetime
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

# --- 1. 爬取景點開放資料 (JSON格式) 並存成JSON檔案 ---
print("-----爬取開放資料並存檔-----")
# 記錄爬取開始時間
start_time_crawl = datetime.datetime.now()
print(f"開始:\n{start_time_crawl}")

# 景點開放資料的 API 網址
# 根據說明網址 (https://data.gov.tw/dataset/7777) 找到的實際資料連結
data_url = "https://media.taiwan.net.tw/XMLReleaseALL_public/scenic_spot_C_f.json"
# 設定 JSON 檔案的名稱
json_file_name = "attractions.json"

try:
    # 發送 HTTP GET 請求，取得 JSON 資料
    response = requests.get(data_url)
    # 檢查 HTTP 請求是否成功 (狀態碼 200 表示成功)
    response.raise_for_status()

    # 將取得的 JSON 資料解析成 Python 物件
    data = response.json()

    # 將資料寫入 JSON 檔案
    with open(json_file_name, "w", encoding="utf-8") as f:
        # 使用 json.dump 將 Python 物件寫入檔案，ensure_ascii=False 確保中文正常顯示
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"資料已成功爬取並儲存到 {json_file_name}")

except requests.exceptions.RequestException as e:
    print(f"爬取資料時發生錯誤: {e}")
except json.JSONDecodeError as e:
    print(f"解析 JSON 資料時發生錯誤: {e}")
except Exception as e:
    print(f"發生未知錯誤: {e}")

# 記錄爬取結束時間
end_time_crawl = datetime.datetime.now()
print(f"結束:\n{end_time_crawl}")
print("=" * 30)

# --- 2. 讀取上述已下載的檔案資料並列出景點總數與前10個景點的名稱 (Name) 與所在縣市 (Region) ---
# 讀取剛剛下載的 JSON 檔案
attractions_data = None # 初始化為 None，以防檔案讀取失敗
try:
    with open(json_file_name, "r", encoding="utf-8") as f:
        # 使用 json.load 讀取 JSON 檔案內容
        attractions_data = json.load(f)

    # 檢查資料結構是否符合預期
    if 'XML_Head' in attractions_data and \
       'Infos' in attractions_data['XML_Head'] and \
       'Info' in attractions_data['XML_Head']['Infos']:

        # 取得景點列表
        attractions_list = attractions_data['XML_Head']['Infos']['Info']

        # 計算景點總數
        total_attractions = len(attractions_list)
        print(f"景點總數:\n{total_attractions}")
        print("只列出前10個景點的名稱")
        print("(Name) 與所在縣市 (Region)")

        # 列出前10個景點的名稱 (Name) 與所在縣市 (Region)
        for i, attraction in enumerate(attractions_list):
            if i >= 10:
                break  # 只列出前10個
            name = attraction.get('Name', 'N/A')  # 使用 .get 避免 key 不存在時報錯
            region = attraction.get('Region', 'N/A')
            print(f"Name: {name};")
            print(f"Region: {region}")
    else:
        print("錯誤: JSON 檔案結構不符預期，找不到 'XML_Head' 或其子鍵。")

except FileNotFoundError:
    print(f"錯誤: 檔案 {json_file_name} 不存在，請確認是否已成功爬取並儲存。")
except json.JSONDecodeError as e:
    print(f"解析 JSON 資料時發生錯誤: {e}")
except Exception as e:
    print(f"讀取或處理檔案時發生未知錯誤: {e}")

print("=" * 30)

# --- 3. 讀取上述已下載的檔案資料後存入MongoDB資料庫 ---
print("-----讀取檔案資料並寫入MongoDB-----")
# 記錄資料庫寫入開始時間
start_time_db_write = datetime.datetime.now()
print(f"資料庫寫入開始:\n{start_time_db_write}")

client = None # 初始化 client 為 None
try:
    # 連接到 MongoDB 伺服器
    # 預設連接到本地端的 MongoDB (mongodb://localhost:27017/)
    client = MongoClient('mongodb://localhost:27017/')
    # 測試連線是否成功
    client.admin.command('ping')
    print("成功連接到 MongoDB。")

    # 選擇資料庫 (如果不存在會自動建立)
    db = client['travel_attractions']
    # 選擇集合 (相當於關聯式資料庫的表格，如果不存在會自動建立)
    collection = db['attractions']

    # 刪除集合中所有現有文件，避免重複寫入 (可選，但為了示範清空)
    # 如果您希望保留舊資料，請註解掉下一行
    collection.delete_many({})
    print("已清空現有集合資料 (如果存在)。")

    # 讀取 JSON 檔案中的資料，並將其插入 MongoDB
    # 我們假設 attractions_data 已經在步驟2中成功加載
    if attractions_data and 'XML_Head' in attractions_data and \
       'Infos' in attractions_data['XML_Head'] and \
       'Info' in attractions_data['XML_Head']['Infos']:

        data_to_insert = attractions_data['XML_Head']['Infos']['Info']
        if data_to_insert: # 確保有資料可以插入
            # 插入多個文件到集合中
            result = collection.insert_many(data_to_insert)
            inserted_count = len(result.inserted_ids)
            print(f"成功寫入\n{inserted_count} 筆資料")
        else:
            print("景點資料列表為空，沒有資料可寫入資料庫。")
    else:
        print("沒有讀取到有效的景點資料，無法寫入資料庫。")

except ConnectionFailure as e:
    print(f"無法連接到 MongoDB 資料庫，請確認 MongoDB 伺服器是否正在運行，或連線字串是否正確: {e}")
except Exception as e:
    print(f"寫入 MongoDB 時發生錯誤: {e}")
finally:
    # 確保關閉 MongoDB 連線
    if client:
        client.close()
        print("MongoDB 連線已關閉。")

# 記錄資料庫寫入結束時間
end_time_db_write = datetime.datetime.now()
print(f"資料庫寫入結束:\n{end_time_db_write}")
print("=" * 30)

-----爬取開放資料並存檔-----
開始:
2025-05-29 10:40:39.768799
資料已成功爬取並儲存到 attractions.json
結束:
2025-05-29 10:40:49.928929
景點總數:
5069
只列出前10個景點的名稱
(Name) 與所在縣市 (Region)
Name: 宏亞食品巧克力觀光工廠;
Region: 桃園市
Name: 臺灣菸酒(股)公司林口觀光酒廠;
Region: 桃園市
Name: 邱良功古厝;
Region: 金門縣
Name: 陳詩吟洋樓;
Region: 金門縣
Name: 朱子祠;
Region: 金門縣
Name: 黃偉墓;
Region: 金門縣
Name: 黃宣顯六路大厝;
Region: 金門縣
Name: 莒光樓;
Region: 金門縣
Name: 奎閣(魁星樓);
Region: 金門縣
Name: 瓊林蔡氏家廟;
Region: 金門縣
-----讀取檔案資料並寫入MongoDB-----
資料庫寫入開始:
2025-05-29 10:40:50.097504
無法連接到 MongoDB 資料庫，請確認 MongoDB 伺服器是否正在運行，或連線字串是否正確: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 683839b26c5692d75bd81704, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000

In [ ]:
pip install requests pymongo